In [35]:
import pandas as pd
from path import Path
import os
import shutil
import fastai
from fastai.vision import *
from fastai.metrics import fbeta, Precision, Recall, accuracy_thresh
from sklearn.metrics import roc_auc_score, recall_score

In [36]:
path_raw_data = Path('../../data/full/')
files = os.listdir(path_raw_data)

path_data = path_raw_data/'images'

img_files = os.listdir(path_data)

df_labels = pd.concat([
    pd.read_csv("../model_data/train.csv").assign(subset='train'), 
    pd.read_csv("../model_data/dev.csv").assign(subset='valid'),
    pd.read_csv("../model_data/test.csv").assign(subset='test')
                      ],
                     axis=0)

df_labels = df_labels.loc[df_labels['Image Index'].isin(img_files)]

In [38]:
df_labels = df_labels.groupby(['Finding Labels']).apply(lambda x: x.sample(1)).reset_index(drop=True)
df_labels.loc[df_labels['Finding Labels'] == 'No Finding', 'Finding Labels'] = ''
df_labels

Image Index  Patient ID  \
0    00014248_000.png       14248   
1    00029940_006.png       29940   
2    00027415_000.png       27415   
3    00012364_006.png       12364   
4    00015304_000.png       15304   
5    00017138_036.png       17138   
6    00019924_031.png       19924   
7    00019508_013.png       19508   
8    00011702_039.png       11702   
9    00015401_022.png       15401   
10   00019706_010.png       19706   
11   00028890_004.png       28890   
12   00020429_009.png       20429   
13   00015770_033.png       15770   
14   00011402_007.png       11402   
15   00001301_044.png        1301   
16   00018644_003.png       18644   
17   00019979_001.png       19979   
18   00015770_034.png       15770   
19   00028012_001.png       28012   
20   00005653_007.png        5653   
21   00015732_025.png       15732   
22   00009667_002.png        9667   
23   00020429_010.png       20429   
24   00022720_002.png       22720   
25   00021796_004.png       21796   
26   00001301_042.png        1301   
27   00004344_045.png        4344   
28   00014687_002.png       14687   
29   00004007_028.png        4007   
..                ...         ...   
806  00026196_001.png       26196   
807  00027837_001.png       27837   
808  00005448_009.png        5448   
809  00030100_000.png       30100   
810  00002848_000.png        2848   
811  00001836_110.png        1836   
812  00018011_005.png       18011   
813  00026078_009.png       26078   
814  00030183_001.png       30183   
815  00021862_004.png       21862   
816  00022572_063.png       22572   
817  00018366_010.png       18366   
818  00009516_001.png        9516   
819  00022237_002.png       22237   
820  00002061_000.png        2061   
821  00016805_008.png       16805   
822  00021284_000.png       21284   
823  00017611_002.png       17611   
824  00023235_000.png       23235   
825  00006378_000.png        6378   
826  00026398_000.png       26398   
827  00011592_001.png       11592   
828  00027465_004.png       27465   
829  00017236_075.png       17236   
830  00011460_043.png       11460   
831  00026793_008.png       26793   
832  00021377_016.png       21377   
833  00029039_020.png       29039   
834  00028330_003.png       28330   
835  00027093_002.png       27093   

                                        Finding Labels  Atelectasis  \
0                                          Atelectasis            1   
1                             Atelectasis|Cardiomegaly            1   
2               Atelectasis|Cardiomegaly|Consolidation            1   
3         Atelectasis|Cardiomegaly|Consolidation|Edema            1   
4    Atelectasis|Cardiomegaly|Consolidation|Edema|E...            1   
5    Atelectasis|Cardiomegaly|Consolidation|Edema|E...            1   
6    Atelectasis|Cardiomegaly|Consolidation|Edema|E...            1   
7    Atelectasis|Cardiomegaly|Consolidation|Edema|E...            1   
8    Atelectasis|Cardiomegaly|Consolidation|Edema|E...            1   
9      Atelectasis|Cardiomegaly|Consolidation|Effusion            1   
10   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
11   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
12   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
13   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
14   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
15   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
16   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
17   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
18   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
19   Atelectasis|Cardiomegaly|Consolidation|Effusio...            1   
20   Atelectasis|Cardiomegaly|Consolidation|Infiltr...            1   
21                      Atelectasis|Cardiomegaly|Edema            1   
22             Atelectasis|Cardiomegaly|Edema|Effusion            1   
23   Atelec

In [39]:
tfms = get_transforms(do_flip=True, 
                      flip_vert=False, 
                      max_rotate=5,
                      max_zoom=1.03,
                      max_lighting=0.03,
                      max_warp=0.03,
                      p_affine=0.05,
                      p_lighting=0.05,
                      )

basic_block = (ImageList
                .from_df(df_labels.query('subset!="test"'),
                         path=path_data)
                .split_by_files(valid_names=df_labels.query('subset=="valid"')['Image Index'].tolist())
                .label_from_df(label_delim='|', cols='Finding Labels')
              )

db_xrays = (basic_block
            .transform(tfms, 
                       size=64) 
            .databunch(bs=32))

db_xrays.normalize(imagenet_stats)

db_xrays.c

14

In [40]:
def auroc_score(input_, target):
    input_, target = input_.cpu().numpy(), target.cpu().numpy() #.reshape(-1, learn.data.c)
    score = []
    for i in range(14):
        try: 
            score.append(roc_auc_score(target[:,i], input_[:,i]))
        except ValueError:
            score.append(0)
    
    return np.array(score).mean()

#     def __init__(self):
#         pass

class AUROC(Callback):
    _order = -20 #Needs to run before the recorder
    def on_epoch_begin(self, **kwargs): self.output, self.target = [], []
    
    def on_batch_end(self, last_target, last_output, train, **kwargs):
        if not train:
            self.output.append(last_output)
            self.target.append(last_target)
                
    def on_epoch_end(self, last_metrics, **kwargs):
        if len(self.output) > 0:
            output = torch.cat(self.output)
            target = torch.cat(self.target)
            metric = auroc_score(output, target)
            return add_metrics(last_metrics, metric)



In [41]:
from functools import partial

accuracy_20 = partial(accuracy_thresh, thresh=0.2)
accuracy_50 = partial(accuracy_thresh, thresh=0.5)
f2_20 = partial(fbeta, thresh=0.2, beta=2)
f2_50 = partial(fbeta, thresh=0.5, beta=2)
f1_20 = partial(fbeta, thresh=0.2, beta=1)
f1_50 = partial(fbeta, thresh=0.5, beta=1)


model = cnn_learner(data=db_xrays, 
                   base_arch=models.densenet121,
                   metrics=[AUROC(), 
                            fbeta, 
                            accuracy_20, accuracy_50, 
                            f1_20, f1_50, f2_20, f2_50]
                   )


model.fit_one_cycle(cyc_len=2, max_lr=5e-2)

In [45]:
os.mkdir('../model_data/sample/')
os.mkdir('../model_data/sample/images')

In [46]:
for i in df_labels['Image Index'].tolist():
    shutil.copy(path_data/i, f'../model_data/sample/images/{i}')